## 一、为TensorFlow模型提供服务
在训练完一个TensorFlow模型后，我们可以将模型包装在一个小型服务中，该服务的唯一作用是进行预测并让其余基础架构查询（例如通过REST或gRPC API）。这样可以使模型与基础架构的其余部分隔离，从而可以轻松切换模型版本或根据需要扩展服务（独立于基础架构的其余部分），执行A/B测试并确保所有软件组件依赖相同的模型版本。它还简化了测试和开发等。我们可以使用任何所需的技术（例如使用Flask库）创建自己的微服务……

### 1、使用TensorFlow Serving
TF Serving是使用C++编写的非常有效的、经过测试的模型服务器。它可以承受很高的负载，可以为多个模型版本提供服务，还可以查看模型库并自动部署最新版本。

In [5]:
# python版本大于3.5
import sys
assert sys.version_info >= (3,5)

# Scikit-Learn 版本大于 0.2
import sklearn
assert sklearn.__version__ >= "0.20"

try:
    # %tensorflow_version only exists in Colab.
    %tensorflow_version 2.x
except Exception:
    pass

# TensorFlow 版本大于 2.0
import tensorflow as tf
assert tf.__version__ >= "2.0"

if not tf.config.list_physical_devices('GPU'):
    print("No GPU was detected. CNNs can be very slow without a GPU.")
    

from tensorflow import keras

import numpy as np
import os

# 使 notebook 每次有固定的输出
np.random.seed(42)

# 绘制图形
%matplotlib inline
import matplotlib as mpl
import matplotlib.pyplot as plt
mpl.rc('axes', labelsize=14)
mpl.rc('xtick', labelsize=12)
mpl.rc('ytick', labelsize=12)

# 图形的保存路径
PROJECT_ROOT_DIR = os.getcwd()
CHAPTER_ID = "chapter19"
IMAGES_PATH = os.path.join(PROJECT_ROOT_DIR, "images", CHAPTER_ID)
os.makedirs(IMAGES_PATH, exist_ok=True)

def save_fig(fig_id, tight_layout=True, fig_extension="png", resolution=300):
    path = os.path.join(IMAGES_PATH, fig_id + "." + fig_extension)
    print("Saving figure", fig_id)
    if tight_layout:
        plt.tight_layout()
    plt.savefig(path, format=fig_extension, dpi=resolution)

No GPU was detected. CNNs can be very slow without a GPU.


#### 保存和加载模型
TensorFlow提供了一个简单的tf.saved_model.save（）函数，可将模型导出为SavedModel格式。我们所要做的就是为它提供模型，并指定其名称和版本号，然后该函数将保存该模型的计算图及其权重：

In [11]:
(X_train_full, y_train_full), (X_test, y_test) = keras.datasets.mnist.load_data()
X_train_full = X_train_full[..., np.newaxis].astype(np.float32) / 255.
X_test = X_test[..., np.newaxis].astype(np.float32) / 255.
X_valid, X_train = X_train_full[:5000], X_train_full[5000:]
y_valid, y_train = y_train_full[:5000], y_train_full[5000:]
X_new = X_test[:3]

In [12]:
np.random.seed(42)
tf.random.set_seed(42)

model = keras.models.Sequential([
    keras.layers.Flatten(input_shape=[28, 28, 1]),
    keras.layers.Dense(100, activation="relu"),
    keras.layers.Dense(10, activation="softmax")
])

model.compile(loss="sparse_categorical_crossentropy",
             optimizer=keras.optimizers.SGD(learning_rate=1e-2),
             metrics=["accuracy"])
model.fit(X_train, y_train, epochs=10, validation_data=(X_valid, y_valid))

Epoch 1/10


2022-11-08 16:52:22.630633: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


1719/1719 [==============================] - 3s 1ms/step - loss: 0.6809 - accuracy: 0.8263 - val_loss: 0.3683 - val_accuracy: 0.9022
Epoch 2/10
1719/1719 [==============================] - 2s 1ms/step - loss: 0.3511 - accuracy: 0.9017 - val_loss: 0.3008 - val_accuracy: 0.9172
Epoch 3/10
1719/1719 [==============================] - 2s 1ms/step - loss: 0.3017 - accuracy: 0.9146 - val_loss: 0.2661 - val_accuracy: 0.9298
Epoch 4/10
1719/1719 [==============================] - 2s 1ms/step - loss: 0.2716 - accuracy: 0.9232 - val_loss: 0.2448 - val_accuracy: 0.9338
Epoch 5/10
1719/1719 [==============================] - 2s 1ms/step - loss: 0.2489 - accuracy: 0.9299 - val_loss: 0.2249 - val_accuracy: 0.9376
Epoch 6/10
1719/1719 [==============================] - 2s 1ms/step - loss: 0.2301 - accuracy: 0.9349 - val_loss: 0.2112 - val_accuracy: 0.9432
Epoch 7/10
1719/1719 [==============================] - 2s 1ms/step - loss: 0.2144 - accuracy: 0.9391 - val_loss: 0.1962 - val_accuracy: 0.9466
Epo

In [16]:
np.round(model.predict(X_new), 2)

1/1 [==============================] - 0s 22ms/step


array([[0.  , 0.  , 0.  , 0.01, 0.  , 0.  , 0.  , 0.99, 0.  , 0.  ],
       [0.  , 0.  , 0.97, 0.01, 0.  , 0.  , 0.01, 0.  , 0.  , 0.  ],
       [0.  , 0.98, 0.01, 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  ]],
      dtype=float32)

In [18]:
model_version = "0001"
model_name = "my_minst_model"
model_path = os.getcwd()
model_path = os.path.join(model_path, model_name, model_version)
model_path

'/Users/dayao/Github/Architect-CTO-growth/人工智能技术/《机器学习实战：基于Scikit-Learn、Keras和TensorFlow》笔记及练习/my_minst_model/0001'

In [20]:
tf.saved_model.save(model, model_path)

INFO:tensorflow:Assets written to: /Users/dayao/Github/Architect-CTO-growth/人工智能技术/《机器学习实战：基于Scikit-Learn、Keras和TensorFlow》笔记及练习/my_minst_model/0001/assets


In [22]:
for root, dirs, files in os.walk(model_name):
    indent = '    ' * root.count(os.sep)
    print('{}{}/'.format(indent, os.path.basename(root)))
    for filename in files:
        print('{}{}'.format(indent + '    ', filename))

my_minst_model/
    0001/
        saved_model.pb
        variables/
            variables.data-00000-of-00001
            variables.index
        assets/


In [28]:
saved_model = tf.saved_model.load(model_path)
y_pred = saved_model(tf.constant(X_new, dtype=tf.float32))
y_pred

<tf.Tensor: shape=(3, 10), dtype=float32, numpy=
array([[6.5909400e-05, 1.6177931e-06, 1.7964992e-03, 5.3981370e-03,
        2.1434157e-06, 8.9914356e-05, 3.6010011e-08, 9.9224466e-01,
        2.7889972e-05, 3.7313148e-04],
       [2.0404183e-03, 2.6443315e-04, 9.7167248e-01, 9.7855488e-03,
        4.4141235e-08, 2.3230128e-03, 9.7217672e-03, 6.1548859e-09,
        4.1921400e-03, 6.6028164e-08],
       [2.0866146e-05, 9.8244458e-01, 6.8953871e-03, 1.1151463e-03,
        3.0730729e-04, 8.2387862e-04, 1.8202510e-03, 3.2524879e-03,
        3.1251931e-03, 1.9478111e-04]], dtype=float32)>

In [29]:
!saved_model_cli show --dir {model_path}

The given SavedModel contains the following tag-sets:
'serve'
